# Práctica 06: Redes bayesianas

Plantea una mejora o extensión posible de la red bayesiana creada. Justifica cómo estas mejoras permitirían un análisis más detallado y personalizado de las situaciones laborales.


Impacto del Liderazgo: Este nodo nos dice como el empleado percibe y afecta su trabajo diario el liderazgo de sus jefes como:

    - Positivo
    - Neutral
    - Negativo

In [7]:
# Importar las librerías necesarias
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Paso 1: Definir la estructura de la red bayesiana
modelo = BayesianNetwork([
    ('Horas de trabajo', 'Balance trabajo-vida'),
    ('Balance trabajo-vida', 'Satisfacción laboral'),
    ('Liderazgo', 'Satisfacción laboral')  # Nueva relación entre Liderazgo y Satisfacción laboral
])

# Paso 2: Definir las CPTs (Tablas de Probabilidad Condicional)
cpt_h = TabularCPD(variable='Horas de trabajo', variable_card=3,
                   values=[[0.6], [0.3], [0.1]],
                   state_names={'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

cpt_b = TabularCPD(variable='Balance trabajo-vida', variable_card=2,
                   values=[[0.4, 0.7, 0.2], [0.6, 0.3, 0.8]],
                   evidence=['Horas de trabajo'], evidence_card=[3],
                   state_names={'Balance trabajo-vida': ['equilibrado', 'no equilibrado'],
                                'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

# Nueva CPT para Liderazgo (L)
cpt_l = TabularCPD(variable='Liderazgo', variable_card=3,
                   values=[[0.4], [0.3], [0.3]],
                   state_names={'Liderazgo': ['positivo', 'neutral', 'negativo']})


# Nueva CPT para Satisfacción laboral (S), dependiente de B y L
cpt_s = TabularCPD(variable='Satisfacción laboral', variable_card=3,
                   values=[[0.7, 0.05, 0.1, 0.3, 0.1, 0.05],  # Equilibrio y Liderazgo positivo
                           [0.1, 0.6, 0.2, 0.3, 0.5, 0.05],   # Equilibrio y Liderazgo neutral
                           [0.2, 0.35, 0.7, 0.4, 0.4, 0.9]],  # No equilibrio y Liderazgo negativo
                   evidence=['Balance trabajo-vida', 'Liderazgo'], evidence_card=[2, 3],
                   state_names={'Satisfacción laboral': ['satisfecho', 'neutral', 'insatisfecho'],
                                'Balance trabajo-vida': ['equilibrado', 'no equilibrado'],
                                'Liderazgo': ['positivo', 'neutral', 'negativo']})


modelo.add_cpds(cpt_h, cpt_b, cpt_l, cpt_s)

# Verificar la validez del modelo después de agregar todas las CPDs
if modelo.check_model():
    print("El modelo es válido.")
else:
    print("El modelo es inválido.")

# Paso 3: Realizar la inferencia
# Se utiliza VariableElimination para consultar la red y realizar inferencias sobre variables de interés.
# Crear un objeto de inferencia
inferencia = VariableElimination(modelo)


# Consultar la probabilidad de estar satisfecho bajo condiciones específicas
resultado = inferencia.query(variables=['Satisfacción laboral'],
                             evidence={'Horas de trabajo': 'moderadas', 'Balance trabajo-vida': 'equilibrado',
                                       'Liderazgo': 'positivo'})  # Valores específicos para las variables

# Impresión de la tabla como resultado
print(resultado)

El modelo es válido.
+------------------------------------+-----------------------------+
| Satisfacción laboral               |   phi(Satisfacción laboral) |
+====================================+=============================+
| Satisfacción laboral(satisfecho)   |                      0.7000 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(neutral)      |                      0.1000 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(insatisfecho) |                      0.2000 |
+------------------------------------+-----------------------------+
